In [0]:
from pyspark.sql import functions as F

In [0]:
new_data_count = 5
source_table_name = "databricks_carltest.default.bank_accounts_source"
seq_control_table = "databricks_carltest.default.id_sequence_control"

In [0]:
spark.sql(f"TRUNCATE TABLE {source_table_name}")

DataFrame[]

In [0]:
last_id = (
    spark.table(seq_control_table)
    .filter("table_name = 'bank_accounts_source'")
    .select("last_id")
    .collect()[0][0]
)

print(f"Last ID: {last_id}")

Last ID: 565


need to ensure the schema is same as souce table 

In [0]:
df_new = (
    spark.range(1, new_data_count + 1)
    .withColumn(
        "transaction_id",
        F.concat(F.lit("TXN-"), F.lit(last_id) + F.col("id"))
    ).withColumn(
        "account_id",
        F.concat(
            F.lit("ACCT-"),
            F.lpad((F.rand()*100000).cast("int"), 6, "0")
        )
    )
    .withColumn("transaction_date", F.current_date())
    .withColumn("posting_date", F.date_add(F.current_date(), 1))
    .withColumn(
        "amount",
        F.round(F.rand() * 1000, 2).cast("decimal(18,2)")
    )
    .withColumn("description", F.lit("Daily Dummy Load"))
    .drop("id")
)


In [0]:
df_new.display()

transaction_id,account_id,transaction_date,posting_date,amount,description
TXN-566,ACCT-051746,2026-02-12,2026-02-13,130.59,Daily Dummy Load
TXN-567,ACCT-082571,2026-02-12,2026-02-13,216.68,Daily Dummy Load
TXN-568,ACCT-021560,2026-02-12,2026-02-13,831.43,Daily Dummy Load
TXN-569,ACCT-044393,2026-02-12,2026-02-13,649.98,Daily Dummy Load
TXN-570,ACCT-067212,2026-02-12,2026-02-13,796.19,Daily Dummy Load


In [0]:
# overwrite
df_new.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(source_table_name)

print(f"✅ Table overwritten with {new_data_count} fresh daily records.")
display(df_new)


✅ Table overwritten with 5 fresh daily records.


transaction_id,account_id,transaction_date,posting_date,amount,description
TXN-566,ACCT-051746,2026-02-12,2026-02-13,130.59,Daily Dummy Load
TXN-567,ACCT-082571,2026-02-12,2026-02-13,216.68,Daily Dummy Load
TXN-568,ACCT-021560,2026-02-12,2026-02-13,831.43,Daily Dummy Load
TXN-569,ACCT-044393,2026-02-12,2026-02-13,649.98,Daily Dummy Load
TXN-570,ACCT-067212,2026-02-12,2026-02-13,796.19,Daily Dummy Load


In [0]:
spark.sql("TRUNCATE TABLE databricks_carltest.default.bank_accounts_source")

df_new.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("databricks_carltest.default.bank_accounts_source")


In [0]:
%sql

select * from databricks_carltest.default.bank_accounts_source
order by transaction_id

transaction_id,account_id,transaction_date,posting_date,amount,description
TXN-566,ACCT-051746,2026-02-12,2026-02-13,130.59,Daily Dummy Load
TXN-567,ACCT-082571,2026-02-12,2026-02-13,216.68,Daily Dummy Load
TXN-568,ACCT-021560,2026-02-12,2026-02-13,831.43,Daily Dummy Load
TXN-569,ACCT-044393,2026-02-12,2026-02-13,649.98,Daily Dummy Load
TXN-570,ACCT-067212,2026-02-12,2026-02-13,796.19,Daily Dummy Load


In [0]:
new_last_id = last_id + new_data_count

spark.sql(f"""
UPDATE {seq_control_table}
SET last_id = {new_last_id}
WHERE table_name = 'bank_accounts_source'
""")


DataFrame[num_affected_rows: bigint]

In [0]:
spark.sql(f"SELECT * FROM {seq_control_table}").display()

table_name,last_id
bank_accounts_source,570
